Importing the required libraries

In [24]:
import pandas as pd
import numpy as np

Load the dataset

In [25]:
data = pd.read_csv('ENJOYSPORT.csv')

In [26]:
data.head()

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


In [27]:
data.shape

(14, 5)

calculating the entropy

In [28]:
def calculate_entropy(data):
    target_column = data.iloc[:, -1]
    target_counts = target_column.value_counts()
    entropy_value = 0
    for value in target_counts.keys():
        probability = target_counts[value] / sum(target_counts)
        entropy_value -= probability * np.log2(probability)
    return entropy_value

function for calculating information Gain


In [29]:
#c. Create a function for calculating Information Gain"""

def calculate_information_gain(data, attribute):
    initial_entropy = calculate_entropy(data)
    attribute_values = data[attribute].unique()
    attribute_counts = data[attribute].value_counts()
    conditional_entropy = []

    for value in attribute_values:
        subset = data[data[attribute] == value]
        subset_entropy = 0
        for result in data.iloc[:, -1].unique():
            try:
                prob_result = subset.iloc[:, -1].value_counts()[result] / len(subset)
                subset_entropy -= prob_result * np.log2(prob_result)
            except:
                subset_entropy = 0
        conditional_entropy.append(subset_entropy)

    for i in range(len(attribute_values)):
        initial_entropy -= attribute_counts[attribute_values[i]] * conditional_entropy[i] / sum(attribute_counts)

    return initial_entropy

In [30]:
class TreeNode():
    def __init__(self, attribute_name=None, attribute_values=None):
        self.attribute_name = attribute_name
        self.attribute_values = attribute_values

    def __str__(self):
        return self.attribute_name

# Function to select the best attribute based on Information Gain (ID3)
def build_tree_node(data, used_features):
    node = TreeNode()
    max_info_gain = 0
    best_attribute = None
    available_attributes = [attr for attr in data.columns[:-1] if attr not in used_features]  # Ensure target column is excluded

    if available_attributes:
        for attr in available_attributes:
            info_gain = calculate_information_gain(data, attr)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_attribute = attr

        if best_attribute:
            used_features.append(best_attribute)
            node.attribute_name = best_attribute
            node.attribute_values = data[best_attribute].unique()
            return node
        else:
            return None

    return None

In [31]:
# Recursive function to build the decision tree (ID3 algorithm)
def build_decision_tree(data, used_features):
    root = build_tree_node(data, used_features)
    decision_tree = {}

    # If root is found, continue building the tree
    if root is not None:
        branches = []
        for value in root.attribute_values:
            subset = data[data[root.attribute_name] == value]
            if calculate_entropy(subset) == 0:  # If pure subset, it's a decision (leaf node)
                branches.append((value, subset.iloc[:, -1].unique()[0]))
            else:
                subtree = build_decision_tree(subset, used_features)
                branches.append((value, subtree))

        decision_tree[root.attribute_name] = branches

    return decision_tree

In [32]:
# Function to print the ID3 decision tree with attributes and its decisions
def print_decision_tree(tree, level=0):
    if isinstance(tree, dict):
        for attribute, branches in tree.items():
            print(f"{'|   ' * level}{attribute}")  # Print the attribute name
            if isinstance(branches, list):
                for branch in branches:
                    print(f"{'|   ' * (level + 1)}{branch[0]} ->", end=" ")
                    if isinstance(branch[1], dict):
                        print()
                        print_decision_tree(branch[1], level + 2)  # Recursive call for further splits
                    else:
                        print(f"Decision: {branch[1]}")  # Print the decision
    else:
        print(tree)


code for creating the Decision Tree

In [33]:
#d. Write code for creating the Decision Tree"""

# Build the ID3 decision tree
used_features = []
id3_decision_tree = build_decision_tree(data, used_features)

# Print the ID3 decision tree with attributes and decisions
print("Final ID3 Decision Tree:")
print_decision_tree(id3_decision_tree)

Final ID3 Decision Tree:
outlook
|   sunny -> 
|   |   humidity
|   |   |   high -> Decision: no
|   |   |   normal -> Decision: yes
|   overcast -> Decision: yes
|   rainy -> 
|   |   windy
|   |   |   False -> Decision: yes
|   |   |   True -> Decision: no
